In [1]:
import os

from azureml.core import Environment, Experiment, ScriptRunConfig, Workspace
from azureml.core.runconfig import DockerConfiguration
from dotenv import load_dotenv

load_dotenv()
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP")
AZURE_WORKSPACE_NAME = os.getenv("AZURE_WORKSPACE_NAME")


In [2]:
experiment_name = "oc-p8-experiment-1"
model_choices = [
    "unet_xception",
    "deeplab_v3plus",
    "fcn_8",
    "vgg_segnet",
]
augment_choices = [False, True]  # [False, True]
resize_choices = [
    64,
    # 80,
    128,
    # 160,
    256,
    # 320,
    512,
    # 640,
]  # [64, 80, 128, 160, 256, 320, 512, 640, 800, 1024]
batch_size_per_resize = {
    64: 128,
    80: 128,
    128: 64,
    160: 64,
    256: 32,
    320: 32,
    512: 16,
    640: 8,
    800: 4,
    1024: 2,
}

source_directory = "./azureml"
compute_target = "ocp8-cluster-gpu"


# connect to your workspace
ws = Workspace(
    subscription_id=AZURE_SUBSCRIPTION_ID,
    resource_group=AZURE_RESOURCE_GROUP,
    workspace_name=AZURE_WORKSPACE_NAME,
)


try:
    env = Environment.get(workspace=ws, name=experiment_name)
except:
    env = Environment(name=experiment_name)
    env.docker.base_image = None
    env.docker.base_dockerfile = source_directory + "/Dockerfile"
    env.python.user_managed_dependencies = True
    env.register(workspace=ws)


for resize in resize_choices:
    for model in model_choices:
        for augment in augment_choices:
            batch_size = batch_size_per_resize[resize]
            run_config = ScriptRunConfig(
                source_directory=source_directory,
                script="train.py",
                arguments=[
                    "--experiment",
                    experiment_name,
                    "--model",
                    model,
                    "--resize",
                    resize,
                    "--batch",
                    batch_size,
                    "--augment" if augment else "--no-augment",
                ],
                compute_target=compute_target,
                environment=env,
                docker_runtime_config=DockerConfiguration(
                    use_docker=True,
                    arguments=[
                        # see : https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py#azureml-data-filedataset-mount
                        "--cap-add",
                        "SYS_ADMIN",
                        "--device",
                        "/dev/fuse",
                    ],
                ),
            )

            # create an experiment
            exp = Experiment(workspace=ws, name=experiment_name)

            # submit the run configuration to start the job
            run = exp.submit(run_config)
            run.tag("model", model)
            run.tag("augment", augment)
            run.tag("resize", resize)

            print(f"Submitted Run : {run.display_name}")
            print(f"Tags : {run.get_tags()}")


Converting non-string tag to string: (augment: False)
Converting non-string tag to string: (resize: 64)


Submitted Run : silly_cushion_z7938z6v
Tags : {'model': 'unet_xception', 'augment': 'False', 'resize': '64'}


Converting non-string tag to string: (augment: True)
Converting non-string tag to string: (resize: 64)


Submitted Run : lucid_double_hd8fsmsj
Tags : {'model': 'unet_xception', 'augment': 'True', 'resize': '64'}


Converting non-string tag to string: (augment: False)
Converting non-string tag to string: (resize: 64)


...

Converting non-string tag to string: (augment: False)
Converting non-string tag to string: (resize: 64)


Submitted Run : affable_cat_4hkcg2v8
Tags : {'model': 'vgg_segnet', 'augment': 'False', 'resize': '64'}


Converting non-string tag to string: (augment: True)
Converting non-string tag to string: (resize: 64)


Submitted Run : clever_engine_r8jwvyl4
Tags : {'model': 'vgg_segnet', 'augment': 'True', 'resize': '64'}


Converting non-string tag to string: (augment: False)
